In [ ]:
import numpy as np
from collections import Counter, defaultdict
import concurrent.futures

# Define ranks and suits
ranks = np.arange(13)  # Map ranks to integers 0-12 (2 to Ace)
suits = np.arange(4)   # Map suits to integers 0-3 (hearts, diamonds, clubs, spades)

# Create a deck with two identical sets of 52 cards (104 cards total)
deck = np.array([(rank, suit) for rank in ranks for suit in suits] * 2, dtype=[('rank', 'i4'), ('suit', 'i4')])

def print_deck(deck):
    """Print the entire deck to show all cards in both identical sets."""
    for card in deck:
        rank = card['rank']
        suit = card['suit']
        suit_name = ["Hearts", "Diamonds", "Clubs", "Spades"][suit]
        rank_name = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"][rank]
        print(f"{rank_name} of {suit_name}", end=", ")
    print("\n")

def print_hand(hand):
    """Print the hand in a readable 'rank of suit' format."""
    hand_cards = []
    for card in hand:
        rank = card['rank']
        suit = card['suit']
        suit_name = ["Hearts", "Diamonds", "Clubs", "Spades"][suit]
        rank_name = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"][rank]
        hand_cards.append(f"{rank_name} of {suit_name}")
    print("Hand:", ", ".join(hand_cards))

def deal_hand(deck):
    """Shuffle the deck and deal the first 5 cards for a single trial."""
    shuffled_deck = np.random.permutation(deck)  # Shuffle the entire deck randomly
    hand = shuffled_deck[:5]  # Take the first 5 cards after shuffling
    return shuffled_deck, hand

def check_one_pair_case(hand):
    """Classify a One Pair hand into Case 0 or Case 1, and eliminate Flush hands."""
    # Check for Flush: exclude if all cards are of the same suit
    if len(set(card['suit'] for card in hand)) == 1:
        return -1  # This is a Flush, not a valid One Pair

    # Extract ranks and count occurrences
    rank_counts = Counter(card['rank'] for card in hand)

    # Identify ranks that form pairs
    pairs = [rank for rank, count in rank_counts.items() if count == 2]

    # If not exactly one pair, it’s not a One Pair hand
    if len(pairs) != 1:
        return -1

    pair_rank = pairs[0]

    # Case 0: The pair is of the same rank but from different suits
    if len(set(card['suit'] for card in hand if card['rank'] == pair_rank)) == 2:
        return 0

    # Case 1: The pair is of the same rank and the same suit
    if len(set(card['suit'] for card in hand if card['rank'] == pair_rank)) == 1:
        return 1

    return -1  # If it doesn't match any specific case
def run_simulation(num_trials):
    """Run the simulation, count occurrences of each case, and collect examples as they occur."""
    counts = Counter()
    local_examples = defaultdict(list)  # Local storage for examples per worker

    for _ in range(num_trials):
        _, hand = deal_hand(deck)  # Shuffle and deal a fresh hand for each trial
        case = check_one_pair_case(hand)

        if case in [0, 1]:  # Only count valid two-pair cases
            counts[case] += 1
            # Collect up to 10 examples for each case if not already filled
            if len(local_examples[case]) < 10:
                local_examples[case].append(hand)

    return counts, local_examples

def run_simulation_parallel(trials, num_workers=4):
    """Run the simulation in parallel using multiple workers."""
    chunk_size = trials // num_workers
    example_hands = defaultdict(list)  # Store examples globally after combining

    with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
        futures = [executor.submit(run_simulation, chunk_size) for _ in range(num_workers)]
        results = [f.result() for f in concurrent.futures.as_completed(futures)]

    # Combine counts and examples from all workers
    combined_counts = Counter()
    for counts, local_examples in results:
        combined_counts.update(counts)
        # Merge examples up to 10 per case
        for case, hands in local_examples.items():
            if len(example_hands[case]) < 10:
                example_hands[case].extend(hands[:10 - len(example_hands[case])])

    return combined_counts, example_hands

def calculate_one_pair_probability(one_pair_count, total_hands):
    """Calculate the probability of getting a One Pair."""
    return one_pair_count / total_hands

def main():
    # Print the initial unshuffled deck
    print("Initial deck:")
    print_deck(deck)

    # Part 1: Run 100 trials, shuffle each time, and print the shuffled deck and dealt hand
    print("\nRunning 100 trials:")
    example_hands_100 = {"Case 0": [], "Case 1": []}
    for trial in range(100):
        shuffled_deck, hand = deal_hand(deck)  # Shuffle and deal a hand
        print(f"\nTrial {trial + 1}: Shuffled deck")
        print_deck(shuffled_deck)  # Print the entire shuffled deck after shuffling

        # Print the first 5 cards as the dealt hand
        print_hand(hand)

        # Check for a one-pair in the dealt hand
        case = check_one_pair_case(hand)
        print(f"One Pair Case: {case}")

        # Collect examples during the initial 100 trials if needed
        if case in example_hands_100 and len(example_hands_100[case]) < 10:
            example_hands_100[case].append(hand)

    # Part 2: Run full simulation for larger trial counts and calculate total one pair probability
    trial_counts = [100, 1_000, 1_000_0, 10_000_0]
    for trials in trial_counts:
        print(f"\nRunning parallel simulation with {trials} trials...")
        counts, example_hands_large = run_simulation_parallel(trials)

        # Calculate total number of one pair hands (sum of all cases)
        total_one_pair_count = sum(counts[case] for case in [0, 1])

        # Calculate total probability for one pair hands
        total_probability = calculate_one_pair_probability(total_one_pair_count, trials)

        # Print results and total probability for one-pair hands
        print(f"\nResults for {trials} trials: {dict(counts)}")
        print(f"Total probability of one pair in {trials} trials: {total_probability:.8f}")



if __name__ == "__main__":
    main()

Initial deck:
2 of Hearts, 2 of Diamonds, 2 of Clubs, 2 of Spades, 3 of Hearts, 3 of Diamonds, 3 of Clubs, 3 of Spades, 4 of Hearts, 4 of Diamonds, 4 of Clubs, 4 of Spades, 5 of Hearts, 5 of Diamonds, 5 of Clubs, 5 of Spades, 6 of Hearts, 6 of Diamonds, 6 of Clubs, 6 of Spades, 7 of Hearts, 7 of Diamonds, 7 of Clubs, 7 of Spades, 8 of Hearts, 8 of Diamonds, 8 of Clubs, 8 of Spades, 9 of Hearts, 9 of Diamonds, 9 of Clubs, 9 of Spades, 10 of Hearts, 10 of Diamonds, 10 of Clubs, 10 of Spades, J of Hearts, J of Diamonds, J of Clubs, J of Spades, Q of Hearts, Q of Diamonds, Q of Clubs, Q of Spades, K of Hearts, K of Diamonds, K of Clubs, K of Spades, A of Hearts, A of Diamonds, A of Clubs, A of Spades, 2 of Hearts, 2 of Diamonds, 2 of Clubs, 2 of Spades, 3 of Hearts, 3 of Diamonds, 3 of Clubs, 3 of Spades, 4 of Hearts, 4 of Diamonds, 4 of Clubs, 4 of Spades, 5 of Hearts, 5 of Diamonds, 5 of Clubs, 5 of Spades, 6 of Hearts, 6 of Diamonds, 6 of Clubs, 6 of Spades, 7 of Hearts, 7 of Diamonds, 